## Physics-Informed Neural Network for Lorenz 96 System
This notebook demonstrates how to build a Physics-Informed Neural Network (PINN) for the Lorenz 96 system, combining physics-based modeling with data-driven learning.

In [12]:

import numpy as np

class RC_ESN:
    def __init__(self, input_dim, reservoir_size, output_dim, spectral_radius=0.95, sparsity=0.1):
        self.W_res = np.random.rand(reservoir_size, reservoir_size) - 0.5
        eigenvalues = np.linalg.eigvals(self.W_res)
        self.W_res *= spectral_radius / max(abs(eigenvalues))
        self.W_in = np.random.rand(reservoir_size, input_dim) * 2 - 1
        mask = np.random.rand(reservoir_size, reservoir_size) > sparsity
        self.W_res[mask] = 0
        self.reservoir_size = reservoir_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.state = np.zeros(reservoir_size)
        self.W_out = None

    def _update(self, input_data):
        self.state = np.tanh(np.dot(self.W_in, input_data) + np.dot(self.W_res, self.state))
        return self.state

    def fit(self, X, y, reg_coef=1e-6):
        N = len(X)
        design_matrix = np.zeros((N, self.reservoir_size))
        for i in range(N):
            design_matrix[i] = self._update(X[i])
        self.W_out = np.dot(np.linalg.inv(np.dot(design_matrix.T, design_matrix) + 
                                          reg_coef * np.eye(self.reservoir_size)), 
                            np.dot(design_matrix.T, y))

    def predict(self, X):
        N = len(X)
        output = np.zeros((N, self.output_dim))
        for i in range(N):
            reservoir_state = self._update(X[i])
            output[i] = np.dot(self.W_out.T, reservoir_state)
        return output


### Step 1: Define the Physics Constraints
We start by expressing the Lorenz 96 system as a set of ordinary differential equations (ODEs).

In [13]:
import numpy as np
import tensorflow as tf

def lorenz_96_derivatives_tf(x, F=8):
    N = tf.shape(x)[0]
    dxdt = tf.zeros_like(x)
    for i in range(N):
        dxdt_i = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
        dxdt = tf.tensor_scatter_nd_update(dxdt, [[i]], [dxdt_i])
    return dxdt


### Step 2: Design the Neural Network Architecture
Next, we create a neural network using TensorFlow to predict the derivatives of the Lorenz 96 system.

In [14]:


N = 5

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(N,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(N)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                384       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
esn = RC_ESN(input_dim=N, reservoir_size=100, output_dim=N)

### Step 3: Embed Physics Constraints as Loss Terms
We embed the physics constraints of the Lorenz 96 system as loss terms in the neural network.

In [16]:
def physics_loss(y_true, y_pred):
    predicted_derivatives = lorenz_96_derivatives_tf(y_pred)
    return tf.reduce_mean(tf.square(predicted_derivatives - y_true))

def combined_loss(y_true, y_pred):
    mse_loss = tf.reduce_mean(tf.square(y_pred - y_true))
    return mse_loss + physics_loss(y_true, y_pred)

model.compile(optimizer='adam', loss=combined_loss)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                384       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 4: Generate Training Data
We simulate the Lorenz 96 system to generate training data for the neural network.

In [17]:
from scipy.integrate import odeint

F = 8

def L96(x, t):
    return lorenz_96_derivatives_tf(x, F)

x0 = F * np.ones(N)
x0[0] += 0.01
t = np.arange(0.0, 30.0, 0.01)
x = odeint(L96, x0, t)
dxdt = np.array([L96(xi, 0) for xi in x])

X_train = x
y_train = dxdt

### Step 5: Train the Neural Network
We train the neural network using the training data, and the model learns how the Lorenz 96 system behaves.

In [18]:

X_train = x[:-1]
y_train = x[1:] - x[:-1]
esn.fit(X_train, y_train)


### Step 6: Evaluate and Use the Hybrid Model
Finally, we validate and utilize the trained PINN for various tasks such as forecasting or analysis.

In [19]:
# Initial state for test data
x0_test = F * np.ones(N)
x0_test[0] += 0.02  # Slightly different initial condition

# Time points for test data
t_test = np.arange(0.0, 30.0, 0.01)

# Solve the differential equations for test data
x_test = odeint(lorenz_96_derivatives_tf, x0_test, t_test)

# Compute the derivatives for test data
dxdt_test = np.array([lorenz_96_derivatives_tf(xi, 0) for xi in x_test])

# Test data
X_test = x_test
y_test = dxdt_test


In [20]:
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)


94/94 [==============================] - 1s 3ms/step - loss: 14297.6943
Test Loss: 14297.6943359375


In [21]:
predictions = esn.predict(X_test)